<a href="https://colab.research.google.com/github/mvapontes/KEGGAPI.jl/blob/main/examples/Case1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Hardware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2023-08-04 08:46:10 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


In [ ]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


In [ ]:
using Pkg

In [ ]:
Pkg.add("Revise")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("BenchmarkTools")
Pkg.add("FastaIO")
Pkg.add(url="https://github.com/bwbioinfo/KEGGAPI.jl")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Requires ─────────── v1.3.0
   Installed OrderedCollections ─ v1.6.2
   Installed LoweredCodeUtils ─── v2.3.0
   Installed CodeTracking ─────── v1.3.2
   Installed JuliaInterpreter ─── v0.9.23
   Installed Revise ───────────── v3.5.3
    Updating `~/.julia/environments/v1.8/Project.toml`
  [295af30f] + Revise v3.5.3
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [da1fd8a2] + CodeTracking v1.3.2
  [aa1ae85d] + JuliaInterpreter v0.9.23
  [6f1432cf] + LoweredCodeUtils v2.3.0
  [bac558e1] + OrderedCollections v1.6.2
  [ae029012] + Requires v1.3.0
  [295af30f] + Revise v3.5.3
  [8ba89e20] + Distributed
Precompiling project...
  ✓ Requires
  ✓ OrderedCollections
  ✓ CodeTracking
  ✓ JuliaInterpreter
  ✓ LoweredCodeUtils
  ✓ Revise
  6 dependencies successfully precompiled in 21 seconds. 20 already precompiled.
   Resolving package versions...
   Installed TableTraits ─────────

In [ ]:
using Revise
using DataFrames
using CSV
using FastaIO
using BenchmarkTools
using KEGGAPI

# Case 1: From Swissprot ID to Kegg information

### 1. Convert outside Database ID to Kegg ID and vice versa



| Database       | DB Identifier
|:---------------|:-----------------|
|Uniprot ID      | "uniprotid"      |
|NCBI Gene ID    | "ncbi-geneid"    |
|NCBI Protein ID | "ncbi-proteinid" |
|KEGG ID         | "genes"          |
    

### 1.1 Outside identifiers directly use as input

To determine if a protein/gene is in KEGG database, the function conv uses as input the KEGG identifier and the gene of interest with the DB identifier.

Only those outside identifiers with a hit in KEGG database are return

In [ ]:
@time kegg_conv_uniprot = KEGGAPI.conv("genes", "uniprot:A0A072UR65")
DataFrame(
    kegg_conv_uniprot.data,
    kegg_conv_uniprot.colnames
    )

  0.535926 seconds (226 allocations: 12.391 KiB)


Row,Target ID,Source ID
,String,String
1,up:A0A072UR65,mtr:25493984


### 1.2 Outside database identifiers from a file as input

Several identifiers from the same database can be run at once. Either as input from a file or several identifiers join by "+" sign.

Only those outside identifiers with a hit in KEGG database are return.

The selected dataset belong to Uniprot proteins Review dataset. User can download the data and upload it to their session. https://www.kaggle.com/datasets/andreylovyagin/uniprot-proteins-reviewed-swissprot?select=data.csv

In [ ]:
df = DataFrame(CSV.File("subset_data.csv", header=1, delim=","))

49×112 DataFrame
 Row │ Column1  Entry       Reviewed  Entry Name   Protein names               ⋯
     │ Int64    String15    String15  String15     String                      ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │       0  A0A024B7W1  reviewed  POLG_ZIKVF   Genome polyprotein [Cleaved ⋯
   2 │       1  A0A024SC78  reviewed  CUTI1_HYPJR  Cutinase, EC 3.1.1.74
   3 │       2  A0A024SH76  reviewed  GUX2_HYPJR   Exoglucanase 2, EC 3.2.1.91
   4 │       3  A0A026W182  reviewed  ORCO_OOCBI   Odorant receptor coreceptor
   5 │       4  A0A044RE18  reviewed  BLI_ONCVO    Endoprotease bli, EC 3.4.21 ⋯
   6 │       5  A0A059TC02  reviewed  CCR1_PETHY   Cinnamoyl-CoA reductase 1,
   7 │       6  A0A060A682  reviewed  HAP2_TETTH   Hapless 2 (Generative cell
   8 │       7  A0A061ACU2  reviewed  PIEZ1_CAEEL  Piezo-type mechanosensitive
   9 │       8  A0A061I403  reviewed  FICD_CRIGR   Protein adenylyltransferase ⋯
  10 │       9  A0A067CMC7  reviewed  HTP3_SAPPC   Endonuclease Htp3, EC 3.1.3
  11 │      10  A0A067XGX8  reviewed  AROG2_PETHY  Phospho-2-dehydro-3-deoxyhe
  ⋮  │    ⋮         ⋮          ⋮           ⋮                       ⋮           ⋱
  40 │      39  A0A0A1C930  reviewed  HMR2A_PANGI  3-hydroxy-3-methylglutaryl
  41 │      40  A0A0A1H8I4  reviewed  AIS_PSESP    Aconitate isomerase, AI, EC ⋯
  42 │      41  A0A0A1HA03  reviewed  708C1_FAGES  UDP-glycosyltransferase 708
  43 │      42  A0A0A1I6E7  reviewed  NDB4S_ANDCR  Antimicrobial peptide AcrAP
  44 │      43  A0A0A1I6N9  reviewed  NDB4T_ANDCR  Antimicrobial peptide AcrAP
  45 │      44  A0A0A7EPL0  reviewed  PIAL1_ARATH  E4 SUMO-protein ligase PIAL ⋯
  46 │      45  A0A0A7EQR3  reviewed  GCE_CERUI    4-O-methyl-glucuronoyl meth
  47 │      46  A0A0A7GEY4  reviewed  GGPPS_GEOAI  Geranylgeranyl pyrophosphat
  48 │      47  A0A0A7HFE1  reviewed  CAS10_STRTR  CRISPR system single-strand
  49 │      48  A0A0A7HIF0  reviewed  CSM3_STRTR   CRISPR system Cms endoribon ⋯
                                                 108 columns and 28 rows omitted

Entry identifiers in sample file belong to Uniprot Database

In [ ]:
db = "uniprot:"
dbentry = string.(db, df.Entry)
entry = join(dbentry, "+")

kegg_conv_uniprot = KEGGAPI.conv("genes", entry)
DataFrame(
  kegg_conv_uniprot.data,
  kegg_conv_uniprot.colnames
)

Row,Target ID,Source ID
,String,String
1,up:A0A024SC78,trr:M419DRAFT_76732
2,up:A0A024SH76,trr:M419DRAFT_122470
3,up:A0A061ACU2,cel:CELE_C10C5.1
4,up:A0A067CMC7,spar:SPRG_03573
5,up:A0A068J840,ag:AIE12479
6,up:A0A072UR65,mtr:25493984
7,up:A0A075BSX9,shz:shn_30305
8,up:A0A075FBG7,ag:AIE77094
9,up:A0A075QQ08,nta:107829212


### 1.3 Convert KEGG identifiers to outside database

To obtain the outside database identifier of a KEGG protein the function conv uses the DB identifier of the desire database and the KEGG gene identifier.

Several identifiers from the same database can be run at once.

Only those identifiers with a hit in the database are return.

In [ ]:
@time ncbi_conv_kegg = KEGGAPI.conv("ncbi-proteinid", "mtr:25493984")
DataFrame(
  ncbi_conv_kegg.data,
  ncbi_conv_kegg.colnames
)

  0.657951 seconds (351 allocations: 82.250 KiB)


Row,Target ID,Source ID
,String,String
1,mtr:25493984,ncbi-proteinid:XP_013458146


### 2. Gene gene information

To obtain gene information at KEGG database the function "find" uses the string "genes" and the KEGG gene identifier.

In [ ]:
@time kegg_find_genes = KEGGAPI.find("genes", "mtr:25493984")
DataFrame(
  kegg_find_genes.data,
  kegg_find_genes.colnames
)

  0.576392 seconds (234 allocations: 13.109 KiB)


Row,ID,Gene Name
,String,String
1,mtr:25493984,class V chitinase CHIT5b


### 3. Get Enzyme sequences, nucleotide and amino acid.

#### 3.1 Get nucleotide sequence and save to fasta file

With the "kegg_get" function user can get nucleotide sequence of one or more gene using an array with KEGG protein id and the string "ntseq".

The output of the function can be save to file using the function FastaWriter from the FastaOI package.

In [ ]:
# Nucleotide sequence
@time kegg_ntseq = KEGGAPI.kegg_get(["mtr:25493984", "shz:shn_30305"], "ntseq")

@time FastaWriter("ntseq.fasta") do fw
    for ch in kegg_ntseq[2]
        write(fw, ch)
    end
end

  0.624419 seconds (362 allocations: 239.680 KiB)


┌ Warning: description line longer than 80 characters (entry 1 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446


  0.040651 seconds (13.35 k allocations: 735.603 KiB, 71.29% compilation time)


#### 3.2 Get amino acid sequence and save to fasta file

With the "kegg_get" function user can get amino acid sequence of one or more gene using an array with KEGG protein id and the string "aaseq".

The output of the function can be save to file using the function FastaWriter from the FastaOI package.

In [ ]:
# Nucleotide sequence
@time kegg_aaseq = KEGGAPI.kegg_get(["mtr:25493984", "shz:shn_30305"], "aaseq")

@time FastaWriter("aaseq.fasta") do fw
    for ch in kegg_aaseq[2]
        write(fw, ch)
    end
end

  0.139374 seconds (228 allocations: 16.734 KiB)
  0.027636 seconds (5.37 k allocations: 275.713 KiB, 96.65% compilation time)


┌ Warning: description line longer than 80 characters (entry 1 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446


### 4. Ortholog group

To identify the ortholog related to the enzyme of interest, the function link takes as input the string "ko", and the KEGG gene identifier.

In [ ]:
@time kegg_ko = KEGGAPI.link("ko", "mtr:25493984")
DataFrame(
  kegg_ko.data,
  kegg_ko.colnames
)

  0.639537 seconds (351 allocations: 82.141 KiB)


Row,Target ID,Source ID
,String,String
1,mtr:25493984,ko:K01183


### 5. Reaction(s) catalyzed by gene of interest

To obtain the reactions associated to a gene, and a KEGG orthogroup, the input of the "link" function are the string "reaction", and KEGG ortholog number as "KXXXXX".

In [ ]:
@time kegg_reaction = KEGGAPI.link("reaction", "K01183")
DataFrame(
  kegg_reaction.data,
  kegg_reaction.colnames
)

  0.170478 seconds (219 allocations: 12.547 KiB)


Row,Target ID,Source ID
,String,String
1,ko:K01183,rn:R01206
2,ko:K01183,rn:R02334


### 6. Reaction information

To obtain the reactions information, the "kegg_get" function requires an array of reaction KEGG identifier as "rn:RXXXXX"

In [ ]:
@time kegg_reaction_info = KEGGAPI.kegg_get(kegg_reaction.data[2])
kegg_reaction_info[2]

  0.136026 seconds (224 allocations: 16.945 KiB)


2-element Vector{String}:
 "ENTRY       R01206             " ⋯ 527 bytes ⋯ " endochitinase B [EC:3.2.1.14]"
 "ENTRY       R02334             " ⋯ 480 bytes ⋯ " endochitinase B [EC:3.2.1.14]"

### 7. Pathway(s) catalyzed by gene of interest

To obtain pathways associated to a gene the input of the "link" function are the string "pathway", and KEGG gene identifier.

In [ ]:
@time kegg_pathways = KEGGAPI.link("pathway", "mtr:25493984")
DataFrame(
  kegg_pathways.data,
  kegg_pathways.colnames
)

  0.180780 seconds (220 allocations: 12.594 KiB)


Row,Target ID,Source ID
,String,String
1,mtr:25493984,path:mtr00520
2,mtr:25493984,path:mtr01100


### 8. Obtain pathway information

To collect information about a pathway, the function find requieres as input the string "pathway" and the KEGG pathway identifer as "path:mapXXXXX".

In [ ]:
@time kegg_pathway_find = KEGGAPI.find("pathway", "path:map00520")
DataFrame(
  kegg_pathway_find.data,
  kegg_pathway_find.colnames
)

  0.218293 seconds (218 allocations: 12.688 KiB)


Row,ID,Pathway
,String,String
1,path:map00520,Amino sugar and nucleotide sugar metabolism


### 9. Download pathway of interest.

The get_image function is to download a any image, the imput is the pathway number as path:mapXXXXX

The save_image function is to save the figure in a png file. The input is a string wiht the name of the file and the extension ".png"

In [ ]:
# save figure pathway
@time kegg_image = KEGGAPI.get_image("path:map00520")
@time KEGGAPI.save_image(kegg_image, "aminoacid.png")

  0.570293 seconds (309 allocations: 360.461 KiB)
  0.016000 seconds (10.67 k allocations: 570.570 KiB, 97.55% compilation time)


"aminoacid.png"

### 10. Visualize saved pathway

In [ ]:
Pkg.add("TestImages")
Pkg.add("Images")
Pkg.add("FileIO")
Pkg.add("Colors")

In [ ]:
using Images, TestImages, Colors
img = load("aminoacid.png")

### 11. Ortholog genes

Identify all genes related to the KEGG ortholog group using the link function. The input is the string "genes" and the KEGG ortholog group as "KXXXXX".

In [ ]:
@time kegg_ko_genes = KEGGAPI.link("genes", "K01183")
DataFrame(
  kegg_ko_genes.data,
  kegg_ko_genes.colnames
)

  2.599635 seconds (58.20 k allocations: 7.468 MiB)


Row,Target ID,Source ID
,String,String
1,ko:K01183,hsa:27159
2,ko:K01183,hsa:1118
3,ko:K01183,ptr:457641
4,ko:K01183,ptr:457114
5,ko:K01183,pps:100977638
6,ko:K01183,pps:100991992
7,ko:K01183,ggo:101143527
8,ko:K01183,ggo:101149661
9,ko:K01183,ggo:101149317


### 12. Save to file ortholog genes sequence for downstream analysis.

#### 12.1 Get nucleotide sequence and save to fasta file

With the "kegg_get" function user can get nucleotide sequence of one or more gene using an array with KEGG protein id and the string "ntseq".

The output of the function can be save to file using the function FastaWriter from the FastaOI package.

In [ ]:
# Nucleotide sequence
@time kegg_ntseq = KEGGAPI.kegg_get(kegg_ko_genes.data[2][1:50], "ntseq")

@time FastaWriter("MSA_ntseq.fasta") do fw
    for ch in kegg_ntseq[2]
        write(fw, ch)
    end
end

  1.822281 seconds (1.47 k allocations: 639.531 KiB)


┌ Warning: description line longer than 80 characters (entry 1 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 2 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 3 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 4 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 5 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 6 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 7 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warn

  0.040468 seconds (13.13 k allocations: 853.924 KiB, 59.39% compilation time)


#### 12.2 Get amino acid sequence and save to fasta file

With the "kegg_get" function user can get amino acid sequence of one or more gene using an array with KEGG protein id and the string "aaseq".
The output of the function can be save to file using the function FastaWriter from the FastaOI package.

In [ ]:
# Nucleotide sequence
@time kegg_aaseq = KEGGAPI.kegg_get(kegg_ko_genes.data[2][1:50], "aaseq")

@time FastaWriter("MSA_aaseq.fasta") do fw
    for ch in kegg_aaseq[2]
        write(fw, ch)
    end
end

  1.354897 seconds (1.33 k allocations: 197.695 KiB)


┌ Warning: description line longer than 80 characters (entry 1 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 2 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 3 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 4 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 5 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 6 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 7 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warn

  0.032210 seconds (13.14 k allocations: 824.627 KiB, 75.70% compilation time)


┌ Warning: description line longer than 80 characters (entry 21 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 22 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 23 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 24 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 25 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 27 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446
┌ Warning: description line longer than 80 characters (entry 28 of FASTA input)
└ @ FastaIO ~/.julia/packages/FastaIO/LA9pk/src/FastaIO.jl:446